## Support Vector Regression


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [24]:
df = pd.read_csv('./data/Ames_Housing_Price_Data.csv', index_col=0)
cleaned = pd.read_csv('./data/cleaned_housing.csv', index_col=0)
loc = pd.read_csv('./data/Ames_loc.csv')
df

,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
PID,,,,,,,,,,,,,,,,,,,,,
909176150,856,126000,30,RL,NaN,7890,Pave,NaN,Reg,Lvl,...,166,0,NaN,NaN,NaN,0,3,2010,WD,Normal
905476230,1049,139500,120,RL,42.0,4235,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal
911128020,1001,124900,30,C (all),60.0,6060,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal
535377150,1039,114000,70,RL,80.0,8146,Pave,NaN,Reg,Lvl,...,111,0,NaN,NaN,NaN,0,5,2009,WD,Normal
534177230,1665,227000,60,RL,70.0,8400,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,11,2009,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903205040,952,121000,30,RL,NaN,8854,Pave,NaN,Reg,Lvl,...,40,0,NaN,NaN,NaN,0,5,2009,WD,Normal
905402060,1733,139600,20,RL,NaN,13680,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,6,2009,WD,Normal
909275030,2002,145000,90,RH,82.0,6270,Pave,NaN,Reg,HLS,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal


In [82]:
housing = pd.read_csv('data/features_dummified_213col.csv')
housing = housing.drop(['YrSold', 'YearBuilt', 'YearRemodAdd'], axis=1)
housing['nb_appreciation'] = cleaned['nb_appreciation'].values
housing['nb_income'] = cleaned['nb_income'].values

raw_housing = housing.copy()
raw_housing.index = df.index
raw_housing.to_csv('./data/dummified_with_PID.csv')
raw_housing

,LotShape,LandSlope,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,HeatingQC,...,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Partial,crime_rate,school_quality,HouseAge,LastRemod,nb_appreciation,nb_income
PID,,,,,,,,,,,,,,,,,,,,,
909176150,4,3,6,6,2,3,3,3,1,3,...,0,0,0,0,5,2,71,60,7,3
905476230,4,3,5,5,3,3,4,3,2,3,...,0,0,0,0,1,4,25,25,6,7
911128020,4,3,5,9,3,3,3,3,1,5,...,0,0,0,0,0,6,77,0,3,6
535377150,4,3,4,8,3,4,2,3,1,4,...,0,0,0,0,0,5,109,6,5,6
534177230,4,3,8,6,3,3,4,3,1,5,...,0,0,0,0,1,6,8,8,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903205040,4,3,6,6,2,3,3,3,1,2,...,0,0,0,0,0,5,93,59,5,6
905402060,3,3,3,5,2,3,0,0,0,5,...,0,0,0,0,1,4,54,54,6,7
909275030,4,3,5,6,2,3,3,3,1,3,...,0,0,0,0,5,2,58,57,7,3


In [81]:
target = df[['SalePrice']]

Standardize all features and target

In [59]:
from sklearn.preprocessing import StandardScaler

continuous_cols = ['GrLivArea', 'LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
                   'TotalBsmtSF', '2ndFlrSF', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 
                   '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MoSold', 'HouseAge', 'LastRemod', 'school_quality', 
                   'crime_rate', 'nb_appreciation', 'nb_income']
housing[continuous_cols] = StandardScaler().fit_transform(housing[continuous_cols])
y = StandardScaler().fit_transform(target)

In [60]:
scaler = StandardScaler()
housing[continuous_cols] = scaler.fit_transform(housing[continuous_cols])
y = scaler.fit_transform(target)
print(y)



[[-0.6939752 ]
 [-0.5140149 ]
 [-0.70863863]
 ...
 [-0.44069775]
 [ 0.52575569]
 [ 0.49242971]]


In [13]:
from sklearn.svm import SVR
svr = SVR()
svr.set_params(kernel='rbf')

SVR()

In [68]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 
X = housing
Y = y[:,0]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [94]:
# grid search
paramDict = {'C': [10], 
             'gamma': [0.0001, 0.0002, 0.0004, 0.0006, 0.0008, 0.001, 0.002, 0.004, 0.006, 0.008, 0.01],
            'epsilon': [0.01, 0.02, 0.04, 0.06, 0.08, 0.1]}
grid = GridSearchCV(svr, paramDict, cv=5, return_train_score = True)
ans_svr = grid.fit(X_train, Y_train) 

In [287]:
for C in [1, 10, 100, 200, 400, 500, 600, 800, 1000]:
    for g in [0.0001, 0.0002, 0.0004, 0.0006, 0.0008, 0.001, 0.002, 0.004, 0.006, 0.008, 0.01]:
        for ep in [0.01, 0.02, 0.04, 0.06, 0.08, 0.1]:
            svr = SVR(kernel='rbf', C=C, gamma=g, epsilon=ep)
            svr.fit(X_train, Y_train)
            print(f'C:{C}, gamma:{g}, epsilon:{ep}, train:{svr.score(X_train, Y_train)}, test:{svr.score(X_test, Y_test)}')


C:1, gamma:0.0001, epsilon:0.01, train:0.7801670073853278, test:0.7956049677275137
C:1, gamma:0.0001, epsilon:0.02, train:0.7805911370616605, test:0.7958263992346972
C:1, gamma:0.0001, epsilon:0.04, train:0.781847634920989, test:0.7970243574238076
C:1, gamma:0.0001, epsilon:0.06, train:0.7806483149106103, test:0.7958530703085296
C:1, gamma:0.0001, epsilon:0.08, train:0.7789986117570841, test:0.7942275390608458
C:1, gamma:0.0001, epsilon:0.1, train:0.7764964516006191, test:0.7912975966275733
C:1, gamma:0.0002, epsilon:0.01, train:0.8251113042583065, test:0.8425258116686798
C:1, gamma:0.0002, epsilon:0.02, train:0.8257972277226681, test:0.843406848973178
C:1, gamma:0.0002, epsilon:0.04, train:0.8266226706996198, test:0.8443450257696271
C:1, gamma:0.0002, epsilon:0.06, train:0.8268949644596958, test:0.8443793396021368
C:1, gamma:0.0002, epsilon:0.08, train:0.8268643374568827, test:0.8445052570243003
C:1, gamma:0.0002, epsilon:0.1, train:0.82808853312856, test:0.8462383763380796
C:1, gamma

C:10, gamma:0.001, epsilon:0.08, train:0.9265567722030413, test:0.9432427110611974
C:10, gamma:0.001, epsilon:0.1, train:0.9266441553920068, test:0.9429113926734436
C:10, gamma:0.002, epsilon:0.01, train:0.9413118984586213, test:0.946188886279018
C:10, gamma:0.002, epsilon:0.02, train:0.9414311792580903, test:0.9460431244281002
C:10, gamma:0.002, epsilon:0.04, train:0.9414479299871863, test:0.9460809690962164
C:10, gamma:0.002, epsilon:0.06, train:0.941354548819423, test:0.9461192213659446
C:10, gamma:0.002, epsilon:0.08, train:0.9415754538835749, test:0.9464052791624942
C:10, gamma:0.002, epsilon:0.1, train:0.9416935879794706, test:0.9462501323829131
C:10, gamma:0.004, epsilon:0.01, train:0.9641391495104724, test:0.9433906781415388
C:10, gamma:0.004, epsilon:0.02, train:0.9644835019730911, test:0.9435401564091785
C:10, gamma:0.004, epsilon:0.04, train:0.9646491094993749, test:0.9432468831743455
C:10, gamma:0.004, epsilon:0.06, train:0.9648083165385234, test:0.9433762861267214
C:10, ga

C:200, gamma:0.0001, epsilon:0.02, train:0.9156674114786714, test:0.935445305181705
C:200, gamma:0.0001, epsilon:0.04, train:0.9155718796443437, test:0.9351093445543563
C:200, gamma:0.0001, epsilon:0.06, train:0.915711941740216, test:0.9354626265561603
C:200, gamma:0.0001, epsilon:0.08, train:0.9156452297116586, test:0.9357294866537
C:200, gamma:0.0001, epsilon:0.1, train:0.91552015268, test:0.935670296315996
C:200, gamma:0.0002, epsilon:0.01, train:0.9271818977443812, test:0.9457491187336604
C:200, gamma:0.0002, epsilon:0.02, train:0.9269023734472404, test:0.9467297248256165
C:200, gamma:0.0002, epsilon:0.04, train:0.9264380315853702, test:0.9471141909766245
C:200, gamma:0.0002, epsilon:0.06, train:0.9263945974373877, test:0.9472048349343569
C:200, gamma:0.0002, epsilon:0.08, train:0.9265125976755636, test:0.9464026261527154
C:200, gamma:0.0002, epsilon:0.1, train:0.9263396705180128, test:0.946479693545665
C:200, gamma:0.0004, epsilon:0.01, train:0.9366431195076164, test:0.95033005107

C:400, gamma:0.001, epsilon:0.06, train:0.9787862589627476, test:0.9403545121994326
C:400, gamma:0.001, epsilon:0.08, train:0.9792898746037452, test:0.9400432499597052
C:400, gamma:0.001, epsilon:0.1, train:0.9792758181219119, test:0.9389501252865742
C:400, gamma:0.002, epsilon:0.01, train:0.9959161581596105, test:0.9218541855234883
C:400, gamma:0.002, epsilon:0.02, train:0.9958736396163398, test:0.9236361206921688
C:400, gamma:0.002, epsilon:0.04, train:0.995353940710006, test:0.9258656867286228
C:400, gamma:0.002, epsilon:0.06, train:0.9943155573905571, test:0.9259806381865647
C:400, gamma:0.002, epsilon:0.08, train:0.992852688503711, test:0.9264277703041436
C:400, gamma:0.002, epsilon:0.1, train:0.9909897195831373, test:0.926123859630338
C:400, gamma:0.004, epsilon:0.01, train:0.9997437905227728, test:0.9079768418944516
C:400, gamma:0.004, epsilon:0.02, train:0.9994884724665575, test:0.9105016563483798
C:400, gamma:0.004, epsilon:0.04, train:0.9984649970571652, test:0.91545351687117

C:500, gamma:0.01, epsilon:0.1, train:0.9926555811582057, test:0.9204278275838322
C:600, gamma:0.0001, epsilon:0.01, train:0.9260675175161366, test:0.9447486905232657
C:600, gamma:0.0001, epsilon:0.02, train:0.9259154211487349, test:0.945396326186603
C:600, gamma:0.0001, epsilon:0.04, train:0.9255342453610479, test:0.9453136737421041
C:600, gamma:0.0001, epsilon:0.06, train:0.9254659526173694, test:0.9455216923865211
C:600, gamma:0.0001, epsilon:0.08, train:0.9257748920491649, test:0.9447586008622374
C:600, gamma:0.0001, epsilon:0.1, train:0.9254380553686125, test:0.944589545086215
C:600, gamma:0.0002, epsilon:0.01, train:0.9347285523765676, test:0.9501854903806899
C:600, gamma:0.0002, epsilon:0.02, train:0.9348418536747678, test:0.9501665867907624
C:600, gamma:0.0002, epsilon:0.04, train:0.9348294430072756, test:0.9500975662818401
C:600, gamma:0.0002, epsilon:0.06, train:0.9348258000493204, test:0.9500690243745256
C:600, gamma:0.0002, epsilon:0.08, train:0.9349600602383205, test:0.950

C:800, gamma:0.001, epsilon:0.02, train:0.9916449229998149, test:0.9284949381059187
C:800, gamma:0.001, epsilon:0.04, train:0.9914994017268949, test:0.9306073122160823
C:800, gamma:0.001, epsilon:0.06, train:0.9908047129102548, test:0.9310753649166742
C:800, gamma:0.001, epsilon:0.08, train:0.9896982147783543, test:0.9304437000219289
C:800, gamma:0.001, epsilon:0.1, train:0.9882270352619121, test:0.9289325794375455
C:800, gamma:0.002, epsilon:0.01, train:0.9983931865145809, test:0.9143230116173805
C:800, gamma:0.002, epsilon:0.02, train:0.9982758132860833, test:0.9161420563412769
C:800, gamma:0.002, epsilon:0.04, train:0.9975715682910105, test:0.9178425863340514
C:800, gamma:0.002, epsilon:0.06, train:0.9962880357871631, test:0.9190441930412775
C:800, gamma:0.002, epsilon:0.08, train:0.9945107506718203, test:0.9188533115878178
C:800, gamma:0.002, epsilon:0.1, train:0.992175165561751, test:0.9199590086666934
C:800, gamma:0.004, epsilon:0.01, train:0.9998806792474844, test:0.902791337643

C:1000, gamma:0.01, epsilon:0.04, train:0.9985587995294325, test:0.9157756691888121
C:1000, gamma:0.01, epsilon:0.06, train:0.9969674432417032, test:0.9180070507526253
C:1000, gamma:0.01, epsilon:0.08, train:0.9949588728421176, test:0.9196265328754455
C:1000, gamma:0.01, epsilon:0.1, train:0.9926555811582057, test:0.9204278275838322


Best model from for loop: C:600, gamma:0.0004, epsilon:0.1, train:0.9504073442650912, test:0.9500467361285259
C:800, gamma:0.0004, epsilon:0.1, train:0.9545239710034148, test:0.9491909994929694
C:1000, gamma:0.0006, epsilon:0.06, train:0.9749847391925706, test:0.9410311063130842
C:1000, gamma:0.0004, epsilon:0.1, train:0.9581727938931139, test:0.9479489389136259

In [289]:
svr_best = ans_svr.best_estimator_
svr_best.score(X_test, Y_test)

0.9236606987545718

In [290]:
svr_best.score(X_train, Y_train)

0.9355026176696613

In [291]:
svr_best.get_params()

{'C': 1,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.01,
 'gamma': 0.006,
 'kernel': 'rbf',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

For loop found slightly better results from built in best model so I'm using this one

In [127]:
from sklearn.model_selection import train_test_split
X = housing.reset_index()
Y = y[:,0]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
svr.fit(X_train, Y_train)

SVR()

In [128]:
svm = SVR(kernel='rbf', C=600, gamma=0.0004, epsilon=0.1)
svm.fit(X_train, Y_train)
svm.score(X_train, Y_train)

0.9504073442650828

In [129]:
svm.score(X_test, Y_test)

0.950046736128508

Rank feature importance

In [139]:
from sklearn.inspection import permutation_importance

# perform permutation importance
results = permutation_importance(svm, X_test, Y_test, scoring='neg_mean_squared_error')
# get importance
sorted_idx = results.importances_mean.argsort()

In [140]:
feature_rank = {}
features = np.array(housing.columns.values)
# summarize feature importance
for i in sorted_idx[::-1]:
    data = {features[i]: results.importances_mean[i]}
    feature_rank.update(data)

IndexError: index 211 is out of bounds for axis 0 with size 211

In [ ]:
feature_rank

Stack models: RandomForestRegressor, GradientBoostingRegressor and SupportVectorRegressor

In [41]:
from sklearn.ensemble import RandomForestRegressor
RF = RandomForestRegressor(n_estimators=250, max_depth=10, min_samples_leaf=2, min_samples_split=6, n_jobs=1, random_state=0)
RF.fit(X_train, Y_train)
RF.score(X_test, Y_test)

0.9027280138626032

In [43]:
from sklearn.ensemble import GradientBoostingRegressor
gbm = GradientBoostingRegressor(n_estimators=10000, learning_rate=0.01, max_depth=4, max_features='sqrt')
gbm.fit(X_train, Y_train)
gbm.score(X_test, Y_test)

0.9415408598820907

In [45]:
gbm.score(X_train, Y_train)

0.9981128929387564

In [47]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import GradientBoostingRegressor
models = [('svr', SVR(kernel='rbf', C=600, gamma=0.0004, epsilon=0.1)), 
          ('rf', RandomForestRegressor(n_estimators=250, max_depth=10, min_samples_leaf=2, min_samples_split=6, n_jobs=1, random_state=0)), 
          ('gmb', GradientBoostingRegressor(n_estimators=10000, learning_rate=0.01, max_depth=4, max_features='sqrt', random_state=44))]
stacked = StackingRegressor(estimators=models, final_estimator = LinearRegression(), cv=5)
stacked.fit(X_train, Y_train)

In [ ]:
#stacked.score(X_train, Y_train)
stacked.score(X_test, Y_test)

In [ ]:
stacked.score(X_train, Y_train)

In [49]:
# # save model as pickle file
import pickle
# pickle.dump(stacked, open('stacked_model.pkl', 'wb'))

In [52]:
# perform permutation importance
results = permutation_importance(stacked, X_test, Y_test, scoring='neg_mean_squared_error')
# get importance
sorted_idx = results.importances_mean.argsort()

In [54]:
feature_rank2 = {}
features = np.array(housing.columns.values)
# summarize feature importance
for i in sorted_idx[::-1]:
    data = {features[i]: results.importances_mean[i]}
    feature_rank2.update(data)

In [309]:
feature_rank2

{'GrLivArea': 0.12656302706790346,
 'OverallQual': 0.06371791996126872,
 'TotalBsmtSF': 0.03130758511065578,
 'HouseAge': 0.017865462310102158,
 'BsmtFinSF1': 0.01665975652461158,
 'LotArea': 0.01587703717277859,
 'OverallCond': 0.014856779724524472,
 'GarageArea': 0.010586766283838967,
 'LastRemod': 0.005874099399957561,
 'FireplaceQu': 0.00499731619774476,
 'KitchenQual': 0.004753558523318199,
 '2ndFlrSF': 0.004299322709575002,
 'MasVnrArea': 0.004212193507557268,
 'ExterQual': 0.004053574933499825,
 'ScreenPorch': 0.0032376122293944444,
 'BsmtQual': 0.002870695082187606,
 'crime_rate': 0.0026259041225602077,
 'school_quality': 0.0025320752669785174,
 'FullBath': 0.0025298145093981634,
 'BsmtExposure': 0.002439289733337691,
 'nb_appreciation': 0.002368546375064688,
 'Neighborhood_Crawfor': 0.002303930937560039,
 'Neighborhood_StoneBr': 0.0018253573180585848,
 'BsmtFullBath': 0.0016338341780710904,
 'Functional': 0.0015893269678066105,
 'HeatingQC': 0.0011964066904988496,
 'HalfBath':

In [61]:
feature_rank2 = {'GrLivArea': 0.12656302706790346,
 'OverallQual': 0.06371791996126872,
 'TotalBsmtSF': 0.03130758511065578,
 'HouseAge': 0.017865462310102158,
 'BsmtFinSF1': 0.01665975652461158,
 'LotArea': 0.01587703717277859,
 'OverallCond': 0.014856779724524472,
 'GarageArea': 0.010586766283838967,
 'LastRemod': 0.005874099399957561,
 'FireplaceQu': 0.00499731619774476,
 'KitchenQual': 0.004753558523318199,
 '2ndFlrSF': 0.004299322709575002,
 'MasVnrArea': 0.004212193507557268,
 'ExterQual': 0.004053574933499825,
 'ScreenPorch': 0.0032376122293944444,
 'BsmtQual': 0.002870695082187606,
 'crime_rate': 0.0026259041225602077,
 'school_quality': 0.0025320752669785174,
 'FullBath': 0.0025298145093981634,
 'BsmtExposure': 0.002439289733337691,
 'nb_appreciation': 0.002368546375064688,
 'Neighborhood_Crawfor': 0.002303930937560039,
 'Neighborhood_StoneBr': 0.0018253573180585848,
 'BsmtFullBath': 0.0016338341780710904,
 'Functional': 0.0015893269678066105,
 'HeatingQC': 0.0011964066904988496,
 'HalfBath': 0.0009390790238726979,
 'Exterior1st_BrkFace': 0.0009314822836945222,
 'BsmtFinSF2': 0.0008784269332177269,
 'Neighborhood_BrkSide': 0.0008156418840125871,
 'KitchenAbvGr': 0.0007756329963100814,
 'SaleCondition_Partial': 0.0006644348244271056,
 'LotConfig_FR2': 0.0006535990686863836,
 'Condition1_Artery': 0.0006283653456529662,
 'MSZoning_RM': 0.0006107592350807281,
 'LotConfig_CulDSac': 0.0005858264877242683,
 'Neighborhood_NoRidge': 0.0005665271047738451,
 'Foundation_PConc': 0.0005571335965075044,
 'Neighborhood_Edwards': 0.00047066755274491193,
 'LotFrontage': 0.0004664970240479613,
 'CentralAir_N': 0.0004590781088738685,
 'MasVnrType_Stone': 0.0004151781907839727,
 'Neighborhood_Gilbert': 0.0004071272809938034,
 'MSSubClass_60': 0.0003702248875739972,
 'Exterior1st_HdBoard': 0.00036500814191780807,
 'PavedDrive': 0.00036495089122889066,
 'SaleCondition_Family': 0.0003644878508184318,
 'HouseStyle_1.5Fin': 0.00036231017049071363,
 'BsmtCond': 0.00035075964083036427,
 'nb_income': 0.00034444991347477584,
 'LandContour_HLS': 0.0003170806441773769,
 'SaleCondition_Abnorml': 0.00031230950893959694,
 'OpenPorchSF': 0.00029192604076584694,
 'LotConfig_FR3': 0.00028194722109250726,
 'Condition1_Feedr': 0.00026983854619656855,
 'MSSubClass_160': 0.0002578601749635201,
 'Neighborhood_SWISU': 0.00025742403907168113,
 'Exterior1st_MetalSd': 0.00025683837153713944,
 'Neighborhood_Somerst': 0.0002380364219118511,
 'Neighborhood_OldTown': 0.00023670269257691917,
 'MSSubClass_90': 0.0002302751177294754,
 'MSSubClass_120': 0.00022987672387005527,
 'WoodDeckSF': 0.00022163236730437657,
 'BldgType_Duplex': 0.00021394047461982397,
 'LotShape': 0.0002111228970984716,
 'MoSold': 0.00020102495693676004,
 'BedroomAbvGr': 0.00019546686637869208,
 'MSZoning_C (all)': 0.00019099422500139379,
 'GarageFinish': 0.00018217866314463178,
 'Condition1_RRNn': 0.00018166314025811064,
 'Exterior2nd_Stucco': 0.00017710093377167334,
 'Neighborhood_ClearCr': 0.00017119777087534204,
 'Exterior2nd_MetalSd': 0.0001650135984106682,
 'LandSlope': 0.00015404261855137962,
 'Neighborhood_CollgCr': 0.0001503366797075606,
 'GarageType_Detchd': 0.00014708881189681584,
 'Condition1_PosN': 0.00014269547675106465,
 'MSSubClass_70': 0.00013895770913817385,
 'MasVnrType_BrkFace': 0.0001365657998191702,
 'MSZoning_FV': 0.00013375866595335406,
 'SaleType_Con': 0.00013231595100925142,
 'Exterior2nd_CmentBd': 0.00011111541980204364,
 'PoolArea': 0.00010804340574489119,
 'Alley_Pave': 0.00010706640364650799,
 'EnclosedPorch': 9.194147701891836e-05,
 'MSZoning_RH': 9.191062475212286e-05,
 'Neighborhood_MeadowV': 7.782704064168283e-05,
 'RoofStyle_Hip': 7.072075786517945e-05,
 'BldgType_TwnhsE': 6.510211482829847e-05,
 'MSSubClass_30': 6.374086765501691e-05,
 'SaleType_CWD': 6.371098850353057e-05,
 'RoofMatl_Tar&Grv': 6.265973571916716e-05,
 'LandContour_Bnk': 6.050942247709801e-05,
 'Electrical_FuseA': 5.967753444831997e-05,
 'Exterior2nd_AsbShng': 5.957355240713697e-05,
 'Condition2_Feedr': 5.882276785642293e-05,
 'Foundation_Slab': 5.3504707539471505e-05,
 'Foundation_BrkTil': 5.192453147454951e-05,
 'Condition1_PosA': 5.102081016434418e-05,
 'LandContour_Low': 4.611524437940634e-05,
 'Neighborhood_Blueste': 4.586592924566008e-05,
 'LotConfig_Corner': 4.545071971517462e-05,
 'BldgType_Twnhs': 4.490309216562682e-05,
 'Exterior2nd_ImStucc': 4.2385940120040486e-05,
 'Neighborhood_Timber': 4.04214109120643e-05,
 'RoofStyle_Gambrel': 4.022701477412044e-05,
 'HouseStyle_2.5Fin': 3.779065107250568e-05,
 'Condition1_RRAn': 3.757166540426515e-05,
 'MSSubClass_75': 3.5064390055182525e-05,
 'GarageType_CarPort': 3.1343781572613914e-05,
 'Neighborhood_Sawyer': 3.066478655179145e-05,
 'HouseStyle_2Story': 3.0324803578531123e-05,
 'Neighborhood_Blmngtn': 2.9595536871453165e-05,
 'RoofStyle_Flat': 2.6988921144571742e-05,
 'Street_Grvl': 2.6634441318584835e-05,
 'Neighborhood_BrDale': 2.291916825140766e-05,
 'SaleType_COD': 2.0748631780184712e-05,
 'Exterior2nd_Wd Sdng': 2.0275359500987466e-05,
 'Condition2_PosN': 2.0071134531030045e-05,
 'MSZoning_I (all)': 1.8820318637806366e-05,
 'Neighborhood_Mitchel': 1.7665024844536192e-05,
 'GarageType_Basment': 1.5917167182186166e-05,
 'Foundation_Wood': 1.4726246985377611e-05,
 'Neighborhood_NPkVill': 1.279231002334208e-05,
 'HouseStyle_SFoyer': 1.1691270003076626e-05,
 'Neighborhood_IDOTRR': 9.513049408250984e-06,
 'MSSubClass_80': 8.665335539982666e-06,
 'HouseStyle_SLvl': 5.899191145594674e-06,
 'MSSubClass_45': 5.578316377635728e-06,
 'SaleType_New': 4.889151918989476e-06,
 'BldgType_2fmCon': 4.517758871797062e-06,
 'MSSubClass_180': 2.577960409179403e-06,
 'Exterior1st_CBlock': 2.2594560116315223e-06,
 'HouseStyle_2.5Unf': 1.9481905679935574e-06,
 'RoofMatl_WdShake': 1.8296271009632381e-06,
 'MasVnrType_BrkCmn': 1.44804914709723e-06,
 'HouseStyle_1.5Unf': 8.696456584342261e-07,
 'Exterior2nd_CBlock': 6.344212879025312e-07,
 'MiscFeature_Othr': 2.352813709069479e-07,
 'Condition2_Artery': 6.779630026959583e-08,
 'Exterior1st_AsphShn': 4.9470027689568766e-08,
 'SaleType_VWD': 3.96772974017856e-08,
 'Exterior2nd_AsphShn': 2.3252303263388008e-08,
 'Neighborhood_Landmrk': 0.0,
 'SaleCondition_AdjLand': 0.0,
 'Exterior1st_ImStucc': 0.0,
 'Condition2_RRAn': 0.0,
 'Condition2_RRAe': 0.0,
 'Heating_Floor': 0.0,
 'MSZoning_A (agr)': 0.0,
 'Utilities_NoSewr': 0.0,
 'MiscFeature_TenC': 0.0,
 'SaleType_ConLI': 0.0,
 'Neighborhood_GrnHill': 0.0,
 'RoofMatl_Roll': 0.0,
 'Condition2_RRNn': 0.0,
 'RoofMatl_Membran': 0.0,
 'Heating_OthW': 0.0,
 'Condition2_PosA': 0.0,
 'MSSubClass_150': 0.0,
 'RoofMatl_Metal': -5.1745217129195445e-08,
 'Exterior1st_PreCast': -2.9461778481426615e-07,
 'Exterior2nd_PreCast': -2.9461778481426615e-07,
 'Exterior1st_Plywood': -6.099523802413631e-07,
 'Electrical_FuseP': -6.762128011969515e-07,
 'Heating_Wall': -8.929656238484962e-07,
 'Exterior2nd_Stone': -1.8620978088107476e-06,
 'MSSubClass_190': -2.0271300648661008e-06,
 'Neighborhood_Veenker': -2.3112046836470944e-06,
 'RoofStyle_Shed': -2.443822523157113e-06,
 'Condition1_RRNe': -2.7358706265800946e-06,
 'Exterior2nd_Brk Cmn': -2.901786899962844e-06,
 'GarageType_2Types': -3.5646995708210615e-06,
 'MSSubClass_85': -4.430015732669046e-06,
 'MSSubClass_40': -6.996704142100962e-06,
 'Heating_GasW': -7.2501582861234095e-06,
 'Exterior2nd_Plywood': -7.740968182992635e-06,
 'Alley_Grvl': -8.792627690547283e-06,
 'Neighborhood_Greens': -9.769427552733246e-06,
 'GarageQual': -1.1139389557686396e-05,
 'Exterior1st_BrkComm': -1.2054533807685785e-05,
 'SaleType_ConLw': -1.2403004990670096e-05,
 'Neighborhood_NWAmes': -1.2535259849084935e-05,
 'Heating_Grav': -1.5428545957624218e-05,
 'MiscFeature_Shed': -1.608968134815675e-05,
 '3SsnPorch': -1.7285215336688976e-05,
 'MiscFeature_Gar2': -1.769162999895324e-05,
 'Electrical_FuseF': -1.890492631281271e-05,
 'Exterior1st_WdShing': -2.823679879336155e-05,
 'Neighborhood_SawyerW': -3.0435580844767608e-05,
 'SaleType_ConLD': -3.102835353232108e-05,
 'Exterior1st_AsbShng': -3.174425046608698e-05,
 'Exterior2nd_Wd Shng': -3.594123940354743e-05,
 'GarageType_NoGarage': -4.294696760026584e-05,
 'Exterior1st_CemntBd': -5.0727185734336654e-05,
 'Exterior2nd_HdBoard': -5.299881277405355e-05,
 'RoofMatl_WdShngl': -5.4115139144272906e-05,
 'Neighborhood_NridgHt': -6.257393170931851e-05,
 'Exterior1st_Wd Sdng': -6.365503627668023e-05,
 'Exterior2nd_BrkFace': -7.380623663685165e-05,
 'Exterior1st_Stucco': -7.653431775370423e-05,
 'GarageType_BuiltIn': -7.82600505781897e-05,
 'SaleCondition_Alloca': -8.363144523564764e-05,
 'Condition1_RRAe': -9.043632860742807e-05,
 'Foundation_Stone': -9.266219551889026e-05,
 'MSSubClass_50': -0.0001156550691771993,
 'SaleType_Oth': -0.00012173015630079082,
 'BsmtHalfBath': -0.0001265793250491004,
 'Fence': -0.0001491839103406209,
 'RoofStyle_Mansard': -0.0001599934860387897,
 'BsmtUnfSF': -0.0001812434881685726,
 'ExterCond': -0.00026159701848952054}

In [62]:
feats = []
for idx, key in enumerate(feature_rank2):
    if abs(feature_rank2[key]) < 1e-4:
        continue
    else:
        feats.append((feature_rank2[key], key))
        
sorted(feats, reverse=True)

[(0.12656302706790346, 'GrLivArea'),
 (0.06371791996126872, 'OverallQual'),
 (0.03130758511065578, 'TotalBsmtSF'),
 (0.017865462310102158, 'HouseAge'),
 (0.01665975652461158, 'BsmtFinSF1'),
 (0.01587703717277859, 'LotArea'),
 (0.014856779724524472, 'OverallCond'),
 (0.010586766283838967, 'GarageArea'),
 (0.005874099399957561, 'LastRemod'),
 (0.00499731619774476, 'FireplaceQu'),
 (0.004753558523318199, 'KitchenQual'),
 (0.004299322709575002, '2ndFlrSF'),
 (0.004212193507557268, 'MasVnrArea'),
 (0.004053574933499825, 'ExterQual'),
 (0.0032376122293944444, 'ScreenPorch'),
 (0.002870695082187606, 'BsmtQual'),
 (0.0026259041225602077, 'crime_rate'),
 (0.0025320752669785174, 'school_quality'),
 (0.0025298145093981634, 'FullBath'),
 (0.002439289733337691, 'BsmtExposure'),
 (0.002368546375064688, 'nb_appreciation'),
 (0.002303930937560039, 'Neighborhood_Crawfor'),
 (0.0018253573180585848, 'Neighborhood_StoneBr'),
 (0.0016338341780710904, 'BsmtFullBath'),
 (0.0015893269678066105, 'Functional'),

In [64]:
feature_names

['GrLivArea',
 'OverallQual',
 'TotalBsmtSF',
 'HouseAge',
 'BsmtFinSF1',
 'LotArea',
 'OverallCond',
 'GarageArea',
 'LastRemod',
 'FireplaceQu',
 'KitchenQual',
 '2ndFlrSF',
 'MasVnrArea',
 'ExterQual',
 'ScreenPorch',
 'BsmtQual',
 'crime_rate',
 'school_quality',
 'FullBath',
 'BsmtExposure',
 'nb_appreciation',
 'Neighborhood_Crawfor',
 'Neighborhood_StoneBr',
 'BsmtFullBath',
 'Functional',
 'HeatingQC',
 'HalfBath',
 'Exterior1st_BrkFace',
 'BsmtFinSF2',
 'Neighborhood_BrkSide',
 'KitchenAbvGr',
 'SaleCondition_Partial',
 'LotConfig_FR2',
 'Condition1_Artery',
 'MSZoning_RM',
 'LotConfig_CulDSac',
 'Neighborhood_NoRidge',
 'Foundation_PConc',
 'Neighborhood_Edwards',
 'LotFrontage',
 'CentralAir_N',
 'MasVnrType_Stone',
 'Neighborhood_Gilbert',
 'MSSubClass_60',
 'Exterior1st_HdBoard',
 'PavedDrive',
 'SaleCondition_Family',
 'HouseStyle_1.5Fin',
 'BsmtCond',
 'nb_income',
 'LandContour_HLS',
 'SaleCondition_Abnorml',
 'OpenPorchSF',
 'LotConfig_FR3',
 'Condition1_Feedr',
 'MSSu

In [65]:
X

,LandSlope,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,HeatingQC,BsmtFullBath,...,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Partial,crime_rate,school_quality,HouseAge,LastRemod,nb_appreciation,nb_income
LotShape,,,,,,,,,,,,,,,,,,,,,
4,3,6,6,2,3,3,3,1,3,1,...,0,0,0,0,1.103052,-1.396186,1.126260,1.756662,0.708077,-1.271868
4,3,5,5,3,3,4,3,2,3,1,...,0,0,0,0,-0.718016,-0.543301,-0.421375,0.044686,0.233573,1.219719
4,3,5,9,3,3,3,3,1,5,0,...,0,0,0,0,-1.173283,0.309584,1.328126,-1.178154,-1.189937,0.596822
4,3,4,8,3,4,2,3,1,4,0,...,0,0,0,0,-1.173283,-0.116858,2.404741,-0.884673,-0.240930,0.596822
4,3,8,6,3,3,4,3,1,5,1,...,0,0,0,0,-0.718016,0.309584,-0.993327,-0.786845,-1.664441,0.596822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,3,6,6,2,3,3,3,1,2,0,...,0,0,0,0,-1.173283,-0.116858,1.866434,1.707748,-0.240930,0.596822
3,3,3,5,2,3,0,0,0,5,0,...,0,0,0,0,-0.718016,-0.543301,0.554308,1.463180,0.233573,1.219719
4,3,5,6,2,3,3,3,1,3,0,...,0,0,0,0,1.103052,-1.396186,0.688885,1.609921,0.708077,-1.271868


In [69]:
### so we don't have to do the model again...

# lets take the array of coefficients we just made and do a dot product 
# with X to get the fitted values of the X we passed
pred_scaled = X[feature_names].dot(np.array(feature_coefficients))
pred = scaler.inverse_transform(np.array(pred_scaled).reshape(-1, 1))
pred

array([[207266.52654392],
       [202662.45733099],
       [206214.6484487 ],
       ...,
       [225705.20116745],
       [230308.91466228],
       [234131.91389644]])

In [70]:
# now, lets combine with our dataframe to get the full predictions

df2 = target.copy()

target['Predicted'] = pred
target['Delta'] = target['SalePrice'] - target['Predicted']
df2 = target.copy()
df2 = target.sort_values('Delta').merge(loc, on='PID')
df2.drop(['location', 'Latitude', 'Longitude'], axis=1, inplace=True)
df2['Prop_Addr'] = df2['Prop_Addr'].str.replace(', Ames, USA', '')
df2 = df2.merge(cleaned['Neighborhood'], on='PID')
df2.to_csv('./data/Predicted_vs_Sales.csv', index=False)

/var/folders/qp/1k1ym6rd1z34yt05vjspksl40000gn/T/ipykernel_38578/147316811.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target['Predicted'] = pred
/var/folders/qp/1k1ym6rd1z34yt05vjspksl40000gn/T/ipykernel_38578/147316811.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target['Delta'] = target['SalePrice'] - target['Predicted']


,LotShape,LandSlope,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,HeatingQC,...,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Partial,crime_rate,school_quality,HouseAge,LastRemod,nb_appreciation,nb_income
PID,,,,,,,,,,,,,,,,,,,,,
909176150,4,3,6,6,2,3,3,3,1,3,...,0,0,0,0,1.103052,-1.396186,1.126260,1.756662,0.708077,-1.271868
905476230,4,3,5,5,3,3,4,3,2,3,...,0,0,0,0,-0.718016,-0.543301,-0.421375,0.044686,0.233573,1.219719
911128020,4,3,5,9,3,3,3,3,1,5,...,0,0,0,0,-1.173283,0.309584,1.328126,-1.178154,-1.189937,0.596822
535377150,4,3,4,8,3,4,2,3,1,4,...,0,0,0,0,-1.173283,-0.116858,2.404741,-0.884673,-0.240930,0.596822
534177230,4,3,8,6,3,3,4,3,1,5,...,0,0,0,0,-0.718016,0.309584,-0.993327,-0.786845,-1.664441,0.596822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903205040,4,3,6,6,2,3,3,3,1,2,...,0,0,0,0,-1.173283,-0.116858,1.866434,1.707748,-0.240930,0.596822
905402060,3,3,3,5,2,3,0,0,0,5,...,0,0,0,0,-0.718016,-0.543301,0.554308,1.463180,0.233573,1.219719
909275030,4,3,5,6,2,3,3,3,1,3,...,0,0,0,0,1.103052,-1.396186,0.688885,1.609921,0.708077,-1.271868


In [72]:
feature_names = []
feature_coefficients = []

for i in feats:
    feature_coefficients.append(i[0])
    feature_names.append(i[1])
    
np.array(feature_coefficients)

array([ 1.26563027e-01,  6.37179200e-02,  3.13075851e-02,  1.78654623e-02,
        1.66597565e-02,  1.58770372e-02,  1.48567797e-02,  1.05867663e-02,
        5.87409940e-03,  4.99731620e-03,  4.75355852e-03,  4.29932271e-03,
        4.21219351e-03,  4.05357493e-03,  3.23761223e-03,  2.87069508e-03,
        2.62590412e-03,  2.53207527e-03,  2.52981451e-03,  2.43928973e-03,
        2.36854638e-03,  2.30393094e-03,  1.82535732e-03,  1.63383418e-03,
        1.58932697e-03,  1.19640669e-03,  9.39079024e-04,  9.31482284e-04,
        8.78426933e-04,  8.15641884e-04,  7.75632996e-04,  6.64434824e-04,
        6.53599069e-04,  6.28365346e-04,  6.10759235e-04,  5.85826488e-04,
        5.66527105e-04,  5.57133597e-04,  4.70667553e-04,  4.66497024e-04,
        4.59078109e-04,  4.15178191e-04,  4.07127281e-04,  3.70224888e-04,
        3.65008142e-04,  3.64950891e-04,  3.64487851e-04,  3.62310170e-04,
        3.50759641e-04,  3.44449913e-04,  3.17080644e-04,  3.12309509e-04,
        2.91926041e-04,  

In [73]:
## inverting back so we can use this formula for R

# standard scaler works by : 
#### formula: z = (x - u) / s

# so to revert, we need to now solve for x:
#### formula: x = (z * s) + u

# where s is the target standard deviation
# u is the mean of the sample
# and z is the scaled

z = X[feature_names].dot(np.array(feature_coefficients))
s = target['SalePrice'].std()
u = scaler.mean_

x = (z * s) + u
x

PID
909176150    207272.188442
905476230    202667.226709
911128020    206220.106436
535377150    198718.600039
534177230    229663.975734
                 ...      
903205040    207951.858343
905402060    201866.431022
909275030    225714.437492
907192040    230319.043438
906223180    234142.783779
Length: 2580, dtype: float64

In [119]:
np.array(feature_coefficients).std()

0.015166157213180629

In [118]:
(np.array(feature_coefficients) / target['SalePrice'].std() * scaler.scale_).dot(raw_housing[feature_names].iloc[0])

271.1761255521966

In [74]:
z = np.array(feature_coefficients)
s = target['SalePrice'].std()
u = scaler.mean_

(z * s) + u

array([187555.78505114, 182840.44820313, 180408.66547232, 179400.0883551 ,
       179309.62293655, 179250.89465091, 179174.34362313, 178853.95986303,
       178500.36333288, 178434.57733408, 178416.28793022, 178382.20612228,
       178375.66872334, 178363.76739893, 178302.54482835, 178275.01463509,
       178256.64770272, 178249.60762147, 178249.43799437, 178242.6458218 ,
       178237.33787057, 178232.4897039 , 178196.58180388, 178182.21161405,
       178178.87218958, 178149.39095314, 178130.08337798, 178129.51338629,
       178125.53258555, 178120.82175491, 178117.81984448, 178109.47652451,
       178108.66350595, 178106.77019222, 178105.44918657, 178103.57845537,
       178102.13040164, 178101.42559649, 178094.93795503, 178094.62503572,
       178094.06838643, 178090.77452775, 178090.17045922, 178087.40163244,
       178087.01021433, 178087.00591875, 178086.97117632, 178086.8077826 ,
       178085.94113377, 178085.46770806, 178083.41416197, 178083.05617849,
       178081.52678467, 1

In [55]:
final_predictions = pd.DataFrame({'Features': feature_names, 'Coefficients': feature_coefficients})
final_predictions.to_csv('./data/final_predictions.csv', index=False)
final_predictions

,Features,Coefficients
0,GrLivArea,0.126563
1,OverallQual,0.063718
2,TotalBsmtSF,0.031308
3,HouseAge,0.017865
4,BsmtFinSF1,0.016660
...,...,...
86,BsmtHalfBath,-0.000127
87,Fence,-0.000149
88,RoofStyle_Mansard,-0.000160
89,BsmtUnfSF,-0.000181


In [126]:
final_predictions.head(10)

,Features,Coefficients
0,GrLivArea,0.126563
1,OverallQual,0.063718
2,TotalBsmtSF,0.031308
3,HouseAge,0.017865
4,BsmtFinSF1,0.016660
5,LotArea,0.015877
6,OverallCond,0.014857
7,GarageArea,0.010587
8,LastRemod,0.005874
9,FireplaceQu,0.004997
